### Import Dependencis

In [1]:
# import urllib3
import pandas as pd
import tldextract
import datetime;
import csv
import requests
import io
import time
import pandas as pd
import os
from os import listdir
import getpass
import sys 
from os.path import isfile, join,getmtime
import json
import urllib
import urllib.parse
import urllib.request

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import psycopg2 
import io
import sql  # the patched version (file is named sql.py)

### Get Data URL & File

In [2]:
#!/usr/bin/env python
# coding: utf-8

# In[246]:


# import urllib3
import pandas as pd
import tldextract
import datetime;
import csv
import requests
import io
import time
import pandas as pd
import os
from os import listdir
import getpass
import sys 
from os.path import isfile, join,getmtime
# http = urllib3.PoolManager()


# In[247]:
username="c-pipelinesec"
password="ayVd&TO^GnuM"

header="$ORIGIN .\n$TTL 3600      ; 1 hour\n{}        IN SOA  need.to.know.only. hostmaster.pl.com. (\n                                {} ; serial\n                                120        ; refresh (2 minutes)\n                                60         ; retry (1 minute)\n                                432000     ; expire (5 days)\n                                60         ; minimum (1 minute)\n                                )\n                        NS      localhost.\n\n"
domainRow="$ORIGIN {}.\n{}                       CNAME   .\n$ORIGIN {}.{}.\n*                       CNAME   .\n\n"
def getResponse(url, isAuth):
    try:
        if isAuth:
            response = requests.get(url,auth=(username,password))
            print(response.content.decode('utf-8'))
        else:
            response = requests.get(url)

        if response.status_code==200:
            return response.content.decode('utf-8')
    except:
        return ""


def getdomains(url,zonename, is_csv=False, headerPos=None, col=None, isAuth=False):
    response=getResponse(url,isAuth)
    if response==" ":
        return
    memo=set()
    if is_csv:  
        file_object = io.StringIO(response)
        df=pd.read_csv(file_object,header=headerPos)
        with open(zonename+".txt", "w") as sources:
            ts = int(time.time())
            sources.write(header.format(zonename,ts))
            for url in df[df[col].notnull()][col]:
                ext=tldextract.extract(url)
                ext=[ext.domain,ext.suffix]
                domain='.'.join(part for part in ext if part)
                if not domain in memo:
                    sources.write(domainRow.format(zonename,domain,domain,zonename))
                    memo.add(domain)

    else:
        lines=response.split("\n")
        with open(zonename+".txt", "w") as sources:
            ts = int(time.time())
            sources.write(header.format(zonename,ts))
            for line in lines :
                if line!="" and not line.startswith("#") :
#                     print(line)
                    ext=tldextract.extract(line)
                    ext=[ext.domain,ext.suffix]
                    domain='.'.join(part for part in ext if part)
                    if not domain in memo:
                        sources.write(domainRow.format(zonename,domain,domain,zonename))
                        memo.add(domain)


# In[248]:


#############################################################################################################


# provide url of text file and filename to parse domains (text file)
# file creates with name zonename.txt and zone will also be added with domains in requisite format
getdomains("https://openphish.com/feed.txt",zonename="openphish.host.pl")


# In[237]:





# In[250]:


getdomains("https://secure.dshield.org/feeds/suspiciousdomains_Low.txt",zonename="secure.dshield.org")


# In[249]:


####### For csv file

# provide url of csv file(in same format as phish_score.csv) and filename to parse domains (csv file)
# file creates with name zonename.txt and zone will also be added with domains in requisite format
getdomains("https://phishstats.info/phish_score.csv",zonename="phishstat.host.pl",is_csv=True,headerPos=7,col="URL")


# In[ ]:

getdomains("https://openphish.com/prvt-ex/feed.csv",zonename="openphishapi.host.pl",is_csv=True,headerPos=0,col="url",isAuth=True)

sector,url,ip,brand,isotime,asn_name,discover_time,family_id,host,country_code,tld,country_name,asn
Social Networking,http://wa2zur5yaq.projects.webpages.one/,138.68.122.195,"Facebook, Inc.",2020-07-15T01:19:11Z,"DigitalOcean, LLC",15-07-2020 01:19:11 UTC,7d715c6e9359d39084550075bf08b5d8,wa2zur5yaq.projects.webpages.one,NL,one,Netherlands,AS14061
Financial,http://tradetengo.com/wp-content/themes/twentytwenty/assets/css/swab/swab/schwab/schwab/home/login.php?cmd=login_submit&id=d531195b19aefb77dda0df02e9f99a35d531195b19aefb77dda0df02e9f99a35&session=d531195b19aefb77dda0df02e9f99a35d531195b19aefb77dda0df02e9f99a35,209.182.205.234,Charles Schwab,2020-07-15T01:18:13Z,"InMotion Hosting, Inc.",15-07-2020 01:18:13 UTC,cbec0406e7318dfb9afc32c4f0413b79,tradetengo.com,US,com,United States of America,AS54641
Retail/Service,https://www.secureupdate.appleld.com.duilawyeryork.com/ap/716e5414b8e87fd/?cmd=_update&dispatch=716e5414b8e87fdb3&locale=_DE,209.59.154.88,Apple Inc.,2020-07-15T01:13:18Z,"Liqu

Converting the file to dictoionary

### Data Preprocessing & EDA Analysis For openphish

In [3]:
###Read CSV File
openphish = pd.read_csv("openphish.csv",encoding = 'utf8', header=0, delimiter= "\t")

In [4]:
###Figureout data with Header
openphish.head()

,"sector,url,ip,brand,isotime,asn_name,discover_time,family_id,host,country_code,tld,country_name,asn"
0,"Retail/Service,https://www.secureupdate.applel..."
1,"Retail/Service,http://secureupdate.appleld.com..."
2,"Retail/Service,http://secureupdate.appleld.com..."
3,"Gaming,""https://services.runescape.com-un.ru/m..."
4,"Financial,http://site9434107.92.webydo.com/,13..."


In [5]:
### Get Dataset information
openphish.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6499 entries, 0 to 6498
Data columns (total 1 columns):
sector,url,ip,brand,isotime,asn_name,discover_time,family_id,host,country_code,tld,country_name,asn    6499 non-null object
dtypes: object(1)
memory usage: 50.9+ KB


In [6]:
### For data shape
openphish.shape

(6499, 1)

In [7]:
### Show all columns in dataset
openphish.columns     

Index(['sector,url,ip,brand,isotime,asn_name,discover_time,family_id,host,country_code,tld,country_name,asn'], dtype='object')

In [8]:
### Describe dataset story
openphish.describe

<bound method NDFrame.describe of      sector,url,ip,brand,isotime,asn_name,discover_time,family_id,host,country_code,tld,country_name,asn
0     Retail/Service,https://www.secureupdate.applel...                                                 
1     Retail/Service,http://secureupdate.appleld.com...                                                 
2     Retail/Service,http://secureupdate.appleld.com...                                                 
3     Gaming,"https://services.runescape.com-un.ru/m...                                                 
4     Financial,http://site9434107.92.webydo.com/,13...                                                 
...                                                 ...                                                 
6494  ,http://bandbsa.be/wp-includes/post/china/logi...                                                 
6495  ,https://www.intesaonline.io/it-persone-e-fami...                                                 
6496  ,http://bandbsa

In [9]:
### Null value arug
openphish.isnull()

,"sector,url,ip,brand,isotime,asn_name,discover_time,family_id,host,country_code,tld,country_name,asn"
0,False
1,False
2,False
3,False
4,False
...,...
6494,False
6495,False
6496,False
6497,False


### Extract URL Column from all

In [37]:
###Extract url of openphish
openphish_url = pd.read_csv("openphish.csv",usecols=(['url']))
openphish_url

,url
0,https://www.secureupdate.appleld.com.duilawyer...
1,http://secureupdate.appleld.com.duilawyeryork....
2,http://secureupdate.appleld.com.duilawyeryork....
3,https://services.runescape.com-un.ru/m=weblogi...
4,http://site9434107.92.webydo.com/
...,...
6494,http://bandbsa.be/wp-includes/post/china/logis...
6495,https://www.intesaonline.io/it-persone-e-famig...
6496,http://bandbsa.be/wp-includes/post/china/logis...
6497,http://bandbsa.be/wp-includes/post/china/logis...


In [38]:
openphish_url.to_csv('openphish_url.csv',index = False)

In [39]:
from tld import get_tld
 
urls_file = "openphish_url.csv"
#URLs should be in column A without a heading, in a CSV file named "urls_file.csv"
 
urls = [line.rstrip('\n') for line in open(urls_file)]
 
for url in urls:
    res = get_tld(url,as_object=True)
    print(res.fld)

TldBadUrl: Is not a valid URL url!

In [13]:
from tld import get_tld
 
urls_file = "openphish_url.csv"
#URLs should be in column A without a heading, in a CSV file named "urls_file.csv"
 
urls = [line.rstrip('\n') for line in open(urls_file)]
the_file = open("domains.csv", "w")
#Create a CSV file within the same file directory and name "domains.csv"
 
the_file.write("root domain, urls \n")
for url in urls:
    the_list = []
    result = get_tld(url,as_object=True).fld
    try:
        root_domain = the_list.append(result)
    except:
        the_list.append("NO ROOT")
    url = the_list.append(url)
    the_list.append("\n")
    the_file.write(",".join(the_list))
the_file.close

TldBadUrl: Is not a valid URL "https://services.runescape.com-un.ru/m=weblogin/loginform356727,955678,44,837,1"!

In [36]:
getdomains("https://openphish.com/prvt-ex/feed.csv",zonename="openphishapi.host.pl",is_csv=True,headerPos=0,col="url",isAuth=True)

sector,url,ip,brand,isotime,asn_name,discover_time,family_id,host,country_code,tld,country_name,asn
Social Networking,http://id128-sync.com/,146.0.76.67,"Facebook, Inc.",2020-07-15T05:18:11Z,HOSTKEY B.V.,15-07-2020 05:18:11 UTC,07306111e75c967ea6396986d4c40434,id128-sync.com,NL,com,Netherlands,AS57043
Financial,https://www.lejouetfrancais.fr/modules/statsregistrations/translations/css/-/bbv/5mzdlzwm=,163.172.34.60,BBVA,2020-07-15T05:12:19Z,ONLINE S.A.S.,15-07-2020 05:12:19 UTC,d142a73daca528a188a5bc655c129f7a,www.lejouetfrancais.fr,FR,fr,France,AS12876
Financial,https://www.lejouetfrancais.fr/modules/statsregistrations/translations/css/-/bbv/5mzdlzwm=/sms.php,163.172.34.60,BBVA,2020-07-15T05:12:19Z,ONLINE S.A.S.,15-07-2020 05:12:19 UTC,d142a73daca528a188a5bc655c129f7a,www.lejouetfrancais.fr,FR,fr,France,AS12876
Financial,https://www.lejouetfrancais.fr/modules/statsregistrations/translations/css/-/bbv/5mzdlzwm=/sms2.php,163.172.34.60,BBVA,2020-07-15T05:12:19Z,ONLINE S.A.S.,15-07-2020 05

In [18]:
from tld import get_tld
 
url = 'https://services.runescape.com-un.ru/m=weblogin/loginform356727,955678,44,837,1' #URL to strip. Change this URL to whatever you want.
 
res = get_tld(url, as_object=True) #Get the root as an object
 
print (res.fld) #res.fld to extract the domain

com-un.ru


In [80]:
final=list(openphish_url['url'].unique())
final

['https://www.secureupdate.appleld.com.duilawyeryork.com/ap/6894a19730d3377?cmd=_update&dispatch=6894a19730d3377f8&locale=_',
 'http://secureupdate.appleld.com.duilawyeryork.com/ap/f135e31d79d0b2a/?cmd=_update&dispatch=f135e31d79d0b2ae1&locale=_',
 'http://secureupdate.appleld.com.duilawyeryork.com/ap/f135e31d79d0b2a?cmd=_update&dispatch=f135e31d79d0b2ae1&locale=_',
 'https://services.runescape.com-un.ru/m=weblogin/loginform356727,955678,44,837,1',
 'http://site9434107.92.webydo.com/',
 'https://southcut.com.au/classes/commons/pdf/Adobe-Log.php',
 'https://secure.runescape.com-un.ru/m=weblogin/loginform690,528,142,72543360,1',
 'https://secure.runescape.com-as.ru/m=weblogin/loginform446,426,361,50849419,1',
 'https://secure.runescape.com-un.ru/m=weblogin/loginform321,491,833,18026158,1',
 'https://secure.runescape.com-en.ru/m=weblogin/loginform846,294,270,27614441,1',
 'https://services.runescape.com-un.ru/m=weblogin/loginform966,515,616,38159175,2497',
 'https://secure.runescape.com-e

In [81]:
!pip install tld

In [69]:
from tld import get_tld
 
urls_file = ["openphish.csv",usecols=(['url']),header=None]
#URLs should be in column A without a heading, in a CSV file named "urls_file.csv"
 
urls = [line.rstrip('\n') for line in open(urls_file)]
 
for url in urls:
    res = get_tld(url,as_object=True)
    print(res.fld)

SyntaxError: invalid syntax (<ipython-input-69-9a3a89585189>, line 3)

### Data Preprocessing & EDA Analysis For phish_score

In [10]:
### Read CSV File
phish_score= pd.read_csv("phish_score.csv",encoding = 'utf8', header=0, delimiter= "\t")

In [11]:
###Figureout data with Header
phish_score.head()

,"Date,Score,URL,IP"
0,"7/8/2020 21:08,5.3,http://sevenqbk.com/,103.26..."
1,"7/8/2020 21:07,2.1,http://owceduamazon.xyz/,19..."
2,"7/8/2020 21:07,2.4,http://ikuhzdswpx.pfirmann-..."
3,"7/8/2020 21:06,4.4,http://biusjkwvhl.pfirmann-..."
4,"7/8/2020 21:06,5.8,http://82.165.27.36/.intesa..."


In [12]:
### Get Dataset information
phish_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96921 entries, 0 to 96920
Data columns (total 1 columns):
 Date,Score,URL,IP    96921 non-null object
dtypes: object(1)
memory usage: 757.3+ KB


In [13]:
### For data shape
phish_score.shape

(96921, 1)

In [14]:
### Show all columns in dataset
phish_score.columns

Index([' Date,Score,URL,IP'], dtype='object')

In [15]:
### Describe dataset story
phish_score.describe

<bound method NDFrame.describe of                                        Date,Score,URL,IP
0      7/8/2020 21:08,5.3,http://sevenqbk.com/,103.26...
1      7/8/2020 21:07,2.1,http://owceduamazon.xyz/,19...
2      7/8/2020 21:07,2.4,http://ikuhzdswpx.pfirmann-...
3      7/8/2020 21:06,4.4,http://biusjkwvhl.pfirmann-...
4      7/8/2020 21:06,5.8,http://82.165.27.36/.intesa...
...                                                  ...
96916  6/8/2020 0:13,7.2,http://secure267.inmotionhos...
96917  6/8/2020 0:11,5.6,http://grp01.id.rakuten.co.j...
96918  6/8/2020 0:11,1.8,http://cyomomir.brandstorey....
96919  6/8/2020 0:10,4.6,http://appapk.store/picpay-c...
96920  6/8/2020 0:10,2.3,http://amzoeson.webstarterz....

[96921 rows x 1 columns]>

In [16]:
### Null value arug
phish_score.isnull()

,"Date,Score,URL,IP"
0,False
1,False
2,False
3,False
4,False
...,...
96916,False
96917,False
96918,False
96919,False


In [14]:
###Extract url of openphish
phish_score_url = pd.read_csv("phish_score.csv",usecols=(['URL']))
phish_score_url

,URL
0,http://sevenqbk.com/
1,http://owceduamazon.xyz/
2,http://ikuhzdswpx.pfirmann-bau.de/img/8ee7e0a3...
3,http://biusjkwvhl.pfirmann-bau.de/.intesa136/?...
4,http://82.165.27.36/.intesa125/?abuse@alice.it
...,...
96916,http://secure267.inmotionhosting.com/~jouget5/...
96917,http://grp01.id.rakuten.co.jp.2c293c23ea49c088...
96918,http://cyomomir.brandstorey.su
96919,http://appapk.store/picpay-continuar.com.br/lo...


In [34]:
phish_score_url.to_csv('phish_score_url.csv',index = False)

In [35]:
from tld import get_tld
 
urls_file = "phish_score_url.csv"
#URLs should be in column A without a heading, in a CSV file named "urls_file.csv"
 
urls = [line.rstrip('\n') for line in open(urls_file)]
 
for url in urls:
    res = get_tld(url,as_object=True)
    print(res.fld)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 6945: character maps to <undefined>

In [23]:
final=list(phish_score_url['URL'].unique())
final

['http://sevenqbk.com/',
 'http://owceduamazon.xyz/',
 'http://ikuhzdswpx.pfirmann-bau.de/img/8ee7e0a3a9ae1040a9b53725d2caed7c77.png',
 'http://biusjkwvhl.pfirmann-bau.de/.intesa136/?abuse@alice.it',
 'http://82.165.27.36/.intesa125/?abuse@alice.it',
 'http://82.165.27.36/img/8ee7e0a3a9ae1040a9b53725d2caed7c77.png',
 'http://www.ebay.de-p-102817401-iid-103817201.unaux.com/rnqzXvb7x86FkUfA/?email=eonetix@venelin.com',
 'http://www.caixa.crie.me',
 'http://wellsconfirm-account.com/webscr.php?cmd=_login-run&amp;amp',
 'http://vitahaus.com.br/wp-admin/css/colors/blue/Dropbox/g5/main.html?accessToFile=validating&amp;fileAccess=49794&amp;encryptedCookie=ccf1b98f0fb8251506d41f921bf90cd7&amp;u=53722e6c0a46abf5d85c2062b0b67efd&amp;connecting=f439b4d2dcff30cf4fb5b71736005f90&amp;phaseAccess=6e6161b8ced884eb437e83f753b4bdd9&amp;p=3333828e4587014612cfda7e09ddf4fc',
 'http://vitahaus.com.br/wp-admin/css/colors/blue/Dropbox/g5/main.html?accessToFile=validating&amp;fileAccess=3322&amp;encryptedCookie

In [17]:
"""
conn = psycopg2.connect(
    #host="18.139.14.106", #your hostname
    #database="phishdb", #your database name
    #user="phishuser", #your username
    #password="LcqAfdcOpmm10A-3+<2Be4U78TM" #your password
)
"""

DB_HOST = "localhost"
DB_DATABASE = "testdb_name"
DB_USER = "postgres" 
DB_PASS = "123456"

tablename = "sanjoy_name"
#cur = conn.cursor()

def getFiles(allfiles=False):
    # files = [(f.split(".")[0],f) for f in listdir(".") if isfile(f) and not f.startswith('.')]
#     files.remove(('postgres', 'postgres.ipynb'))
    # files.remove(('postgres', 'postgres.py'))
    files=[f for f in listdir(".") if isfile(f) and not f.startswith('.')]
    # files.sort(key=getmtime, reverse=True)
    #['file.csv', 'Last Main.ipynb', 'openphish.csv', 'openphish.host.pl.txt', 'openphishapi.host.pl.txt', 
    # 'phishstat.host.pl.txt', 'phish_score.csv', 'Project.ipynb', 'secure.dshield.org.txt']
    #print(files)
    files.remove('file.csv')
    files.remove('openphish.host.pl.txt')
    files.remove('openphishapi.host.pl.txt')
    files.remove('phishstat.host.pl.txt')
    files.remove('Project.ipynb')
    files.remove('secure.dshield.org.txt')
    files.remove('Last Main.ipynb')
    files.remove('phish_score.csv')
    
    
    files=sorted(files)
    if allfiles:
        return [("_".join(f.replace("-","").split(".")[:-1]),f) for f in files]
        # return [([0].replace("-","")+"_"+[1].replace("-",""),f) for f in files]
    dic={}
    # print(files)
    for f in files:
        fname="_".join(f.replace("-","").split(".")[:-1])
        dic[fname]=f
    return dic;


In [18]:
def getFileContent(filename):
    
    df = pd.read_csv("openphish.csv", header=0)
    data = []
    for x, y in df.iterrows():
        d= y.to_dict()
        data.append(d)
    return data

In [19]:
def createTable(tablename,cur,conn):
    try:
        cmd = "DROP TABLE IF EXISTS {} CASCADE;".format(tablename)
        cur.execute(cmd)
        conn.commit()
        
        cmd="CREATE TABLE IF NOT EXISTS {} (id serial not null, sector VARCHAR(200),url TEXT,ip VARCHAR(250),brand VARCHAR(250),isotime timestamptz,asn_name VARCHAR(250),discover timestamptz,family_id VARCHAR(100),host VARCHAR(150),country_code VARCHAR(250),tld VARCHAR(250),country_name VARCHAR(250),asn VARCHAR(250),PRIMARY KEY (id))".format(tablename)
        cur.execute(cmd)
        conn.commit()
        print("{} table created. ".format(tablename))
    except (Exception, psycopg2.DatabaseError) as error:
        print("createTable: ",error)
        conn.rollback()

In [20]:
def insertTable(tablename,cur,conn,records):
    
    #rec = [('Retail/Service','https://www.secureupdate.appleld.com','209.59.154.88','Apple Inc.','Liquid Web, L.L.C','09-07-2020 03:13:18 UTC','bb09f97a5c884976d54f284a7bdb309c','www.secureupdate.appleld.com.duilawyeryork.com','US','com','United States of America','AS32244'),]
    try:
        print("Inserting in ", tablename)
        i = 0
        for d in records: 
            i = i+ 1
            print(i)
            query = "INSERT INTO "+tablename+" (sector,url,ip,brand,isotime,asn_name,discover,family_id,host,country_code,tld,country_name,asn) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)" 
            cur.execute(query, (d['sector'], d['url'], d['ip'],d['brand'], d['isotime'], d['asn_name'], d['discover_time'], d['family_id'], d['host'], d['country_code'], d['tld'], d['country_name'], d['asn']))
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("insertTable: ",error)
        conn.rollback()

In [21]:
def insertFile(tablename, filepath,cur,conn):
    
    fileRecords = getFileContent(filepath) 
    
    
    print(fileRecords[0])
    createTable(tablename,cur,conn)
    
    insertTable(tablename,cur,conn, fileRecords)

In [22]:
def insertRecords(afiles=False):
    try:
        os.chdir("E:\Projectterm")
        conn = psycopg2.connect(host=DB_HOST,database=DB_DATABASE, user=DB_USER, password=DB_PASS,sslmode="disable")
        cur = conn.cursor()
        allFiles=getFiles(afiles)
        print("Total Distinct Zones: ",len(allFiles))
        print(allFiles)
        if allFiles:
            for file in allFiles:
                print("Working on file ",file[1])
                insertFile(file[0],file[1],cur,conn)
        else:
            for tableName in allFiles.keys():
                print("Working on file ",allFiles[tableName])
                insertFile(tableName,allFiles[tableName],cur,conn)
                cur.close()

    except (Exception, psycopg2.DatabaseError) as error:
        print("insertRecords: ",error)

insertRecords(True)

Total Distinct Zones:  2
[('Data Prob', 'Data Prob.ipynb'), ('openphish', 'openphish.csv')]
Working on file  Data Prob.ipynb
{'sector': 'Retail/Service', 'url': 'https://www.secureupdate.appleld.com.duilawyeryork.com/ap/6894a19730d3377?cmd=_update&dispatch=6894a19730d3377f8&locale=_', 'ip': '209.59.154.88', 'brand': 'Apple Inc.', 'isotime': '2020-07-09T03:13:18Z', 'asn_name': 'Liquid Web, L.L.C', 'discover_time': '09-07-2020 03:13:18 UTC', 'family_id': 'bb09f97a5c884976d54f284a7bdb309c', 'host': 'www.secureupdate.appleld.com.duilawyeryork.com', 'country_code': 'US', 'tld': 'com', 'country_name': 'United States of America', 'asn': 'AS32244'}
createTable:  syntax error at or near "Prob"
LINE 1: DROP TABLE IF EXISTS Data Prob CASCADE;
                                  ^

Inserting in  Data Prob
1
insertTable:  syntax error at or near "Prob"
LINE 1: INSERT INTO Data Prob (sector,url,ip,brand,isotime,asn_name,...
                         ^

Working on file  openphish.csv
{'sector': 'Retail/

1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1752
1753
1754
1755
1756
1757
1758
1759
1760
1761
1762
1763
1764
1765
1766
1767
1768
1769
1770
1771
1772
1773
1774
1775
1776
1777
1778
1779
1780
1781
1782
1783
1784
1785
1786
1787
1788
1789
1790
1791
1792
1793
1794
1795
1796
1797
1798
1799
1800
1801
1802
1803
1804
1805
1806
1807
1808
1809
1810
1811
1812
1813
1814
1815
1816
1817
1818
1819
1820
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832
1833
1834
1835
1836
1837
1838
1839
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941


4021
4022
4023
4024
4025
4026
4027
4028
4029
4030
4031
4032
4033
4034
4035
4036
4037
4038
4039
4040
4041
4042
4043
4044
4045
4046
4047
4048
4049
4050
4051
4052
4053
4054
4055
4056
4057
4058
4059
4060
4061
4062
4063
4064
4065
4066
4067
4068
4069
4070
4071
4072
4073
4074
4075
4076
4077
4078
4079
4080
4081
4082
4083
4084
4085
4086
4087
4088
4089
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4106
4107
4108
4109
4110
4111
4112
4113
4114
4115
4116
4117
4118
4119
4120
4121
4122
4123
4124
4125
4126
4127
4128
4129
4130
4131
4132
4133
4134
4135
4136
4137
4138
4139
4140
4141
4142
4143
4144
4145
4146
4147
4148
4149
4150
4151
4152
4153
4154
4155
4156
4157
4158
4159
4160
4161
4162
4163
4164
4165
4166
4167
4168
4169
4170
4171
4172
4173
4174
4175
4176
4177
4178
4179
4180
4181
4182
4183
4184
4185
4186
4187
4188
4189
4190
4191
4192
4193
4194
4195
4196
4197
4198
4199
4200
4201
4202
4203
4204
4205
4206
4207
4208
4209
4210
4211
4212
4213
4214
4215
4216
4217
4218
4219
4220


6338
6339
6340
6341
6342
6343
6344
6345
6346
6347
6348
6349
6350
6351
6352
6353
6354
6355
6356
6357
6358
6359
6360
6361
6362
6363
6364
6365
6366
6367
6368
6369
6370
6371
6372
6373
6374
6375
6376
6377
6378
6379
6380
6381
6382
6383
6384
6385
6386
6387
6388
6389
6390
6391
6392
6393
6394
6395
6396
6397
6398
6399
6400
6401
6402
6403
6404
6405
6406
6407
6408
6409
6410
6411
6412
6413
6414
6415
6416
6417
6418
6419
6420
6421
6422
6423
6424
6425
6426
6427
6428
6429
6430
6431
6432
6433
6434
6435
6436
6437
6438
6439
6440
6441
6442
6443
6444
6445
6446
6447
6448
6449
6450
6451
6452
6453
6454
6455
6456
6457
6458
6459
6460
6461
6462
6463
6464
6465
6466
6467
6468
6469
6470
6471
6472
6473
6474
6475
6476
6477
6478
6479
6480
6481
6482
6483
6484
6485
6486
6487
6488
6489
6490
6491
6492
6493
6494
6495
6496
6497
6498
6499


In [23]:
allFiles=getFiles(False)
allFiles

{'Data Prob': 'Data Prob.ipynb', 'openphish': 'openphish.csv'}

In [24]:
conn = psycopg2.connect(host=DB_HOST,database=DB_DATABASE, user=DB_USER, password=DB_PASS,sslmode="disable")
cur = conn.cursor()

In [25]:
if allFiles:
   # print(allFiles['Data Prob'])
    
    #    print("Working on file ",file[1])
    insertFile("openphish",allFiles['openphish'],cur,conn)

{'sector': 'Retail/Service', 'url': 'https://www.secureupdate.appleld.com.duilawyeryork.com/ap/6894a19730d3377?cmd=_update&dispatch=6894a19730d3377f8&locale=_', 'ip': '209.59.154.88', 'brand': 'Apple Inc.', 'isotime': '2020-07-09T03:13:18Z', 'asn_name': 'Liquid Web, L.L.C', 'discover_time': '09-07-2020 03:13:18 UTC', 'family_id': 'bb09f97a5c884976d54f284a7bdb309c', 'host': 'www.secureupdate.appleld.com.duilawyeryork.com', 'country_code': 'US', 'tld': 'com', 'country_name': 'United States of America', 'asn': 'AS32244'}
openphish table created. 
Inserting in  openphish
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
13

2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257
2258
2259
2260
2261
2262
2263
2264
2265
2266
2267
2268
2269
2270
2271
2272
2273
2274
2275
2276
2277
2278
2279
2280
2281
2282
2283
2284
2285
2286
2287
2288
2289
2290
2291
2292
2293
2294
2295
2296
2297
2298
2299
2300
2301
2302
2303
2304
2305
2306
2307
2308
2309
2310
2311
2312
2313
2314
2315
2316
2317
2318
2319
2320
2321
2322
2323
2324
2325
2326
2327
2328
2329
2330
2331
2332
2333
2334
2335
2336
2337
2338
2339
2340
2341
2342
2343
2344
2345
2346
2347
2348
2349
2350
2351
2352
2353
2354
2355
2356
2357
2358
2359
2360
2361
2362
2363
2364
2365
2366
2367
2368
2369
2370
2371
2372
2373
2374
2375
2376
2377
2378
2379
2380
2381
2382
2383
2384
2385
2386
2387
2388
2389
2390
2391


4530
4531
4532
4533
4534
4535
4536
4537
4538
4539
4540
4541
4542
4543
4544
4545
4546
4547
4548
4549
4550
4551
4552
4553
4554
4555
4556
4557
4558
4559
4560
4561
4562
4563
4564
4565
4566
4567
4568
4569
4570
4571
4572
4573
4574
4575
4576
4577
4578
4579
4580
4581
4582
4583
4584
4585
4586
4587
4588
4589
4590
4591
4592
4593
4594
4595
4596
4597
4598
4599
4600
4601
4602
4603
4604
4605
4606
4607
4608
4609
4610
4611
4612
4613
4614
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4630
4631
4632
4633
4634
4635
4636
4637
4638
4639
4640
4641
4642
4643
4644
4645
4646
4647
4648
4649
4650
4651
4652
4653
4654
4655
4656
4657
4658
4659
4660
4661
4662
4663
4664
4665
4666
4667
4668
4669
4670
4671
4672
4673
4674
4675
4676
4677
4678
4679
4680
4681
4682
4683
4684
4685
4686
4687
4688
4689
4690
4691
4692
4693
4694
4695
4696
4697
4698
4699
4700
4701
4702
4703
4704
4705
4706
4707
4708
4709
4710
4711
4712
4713
4714
4715
4716
4717
4718
4719
4720
4721
4722
4723
4724
4725
4726
4727
4728
4729


In [26]:
##################################################################
def getFilesAgain(allfilesAgain=False):
    # files = [(f.split(".")[0],f) for f in listdir(".") if isfile(f) and not f.startswith('.')]
#     files.remove(('postgres', 'postgres.ipynb'))
    # files.remove(('postgres', 'postgres.py'))
    files=[f for f in listdir(".") if isfile(f) and not f.startswith('.')]
    # files.sort(key=getmtime, reverse=True)
    #['file.csv', 'Last Main.ipynb', 'openphish.csv', 'openphish.host.pl.txt', 'openphishapi.host.pl.txt', 
    # 'phishstat.host.pl.txt', 'phish_score.csv', 'Project.ipynb', 'secure.dshield.org.txt']
    #print(files)
    files.remove('file.csv')
    files.remove('openphish.host.pl.txt')
    files.remove('openphishapi.host.pl.txt')
    files.remove('phishstat.host.pl.txt')
    files.remove('Project.ipynb')
    files.remove('secure.dshield.org.txt')
    files.remove('Last Main.ipynb')
    files.remove('openphish.csv')
    
    
    files=sorted(files)
    if allfilesAgain:
        return [("_".join(f.replace("-","").split(".")[:-1]),f) for f in files]
        # return [([0].replace("-","")+"_"+[1].replace("-",""),f) for f in files]
    dic={}
    # print(files)
    for f in files:
        fname="_".join(f.replace("-","").split(".")[:-1])
        dic[fname]=f
    return dic;

In [27]:
################################################################
def getFileContentAgain(filename):
    
    df = pd.read_csv("phish_score.csv", header=0)
    data = []
    for x, y in df.iterrows():
        d= y.to_dict()
        data.append(d)
    return data

In [28]:
################################################################
def createTableAgain(tablename,cur,conn):
    try:
        cmd = "DROP TABLE IF EXISTS {} CASCADE;".format(tablename)
        cur.execute(cmd)
        conn.commit()
        
        cmd="CREATE TABLE IF NOT EXISTS {} (id serial not null,Date VARCHAR(20),Score VARCHAR(2),URL VARCHAR(300),IP VARCHAR(30),PRIMARY KEY (id))".format(tablename)
        cur.execute(cmd)
        conn.commit()
        print("{} table created. ".format(tablename))
    except (Exception, psycopg2.DatabaseError) as error:
        print("createTableAgain: ",error)
        conn.rollback()

In [35]:
############################################################
def insertTableAgain(tablename,cur,conn,records):
    
    #rec = [('Retail/Service','https://www.secureupdate.appleld.com','209.59.154.88','Apple Inc.','Liquid Web, L.L.C','09-07-2020 03:13:18 UTC','bb09f97a5c884976d54f284a7bdb309c','www.secureupdate.appleld.com.duilawyeryork.com','US','com','United States of America','AS32244'),]
    try:
        print("Inserting in ", tablename)
        i = 0
        for d in records: 
            i = i+ 1
            print(i)
            query = "INSERT INTO "+tablename+" (Date,Score,URL,IP) VALUES (%s,%s,%s,%s)" 
            cur.execute(query, (d['Date'], d['Score'], d['URL'],d['IP']))
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("insertTableAgain: ",error)
        conn.rollback()

In [36]:
####################################################################
def insertFileAgain(tablename, filepath,cur,conn):
    
    fileRecords = getFileContentAgain(filepath) 
    
    
    print(fileRecords[0])
    createTableAgain(tablename,cur,conn)
    
    insertTableAgain(tablename,cur,conn, fileRecords)

In [37]:
###########################################################################
def insertRecordsAgain(afiles=False):
    try:
        os.chdir("E:\Projectterm")
        conn = psycopg2.connect(host=DB_HOST,database=DB_DATABASE, user=DB_USER, password=DB_PASS,sslmode="disable")
        cur = conn.cursor()
        allFilesAgain=getFilesAgain(afiles)
        print("Total Distinct Zones: ",len(allFilesAgain))
        print(allFilesAgain)
        if allFilesAgain:
            for file in allFilesAgain:
                print("Working on file ",file[1])
                insertFileAgain(file[0],file[1],cur,conn)
        else:
            for tableName in allFilesAgain.keys():
                print("Working on file ",allFilesAgain[tableName])
                insertFileAgain(tableName,allFilesAgain[tableName],cur,conn)
                cur.close()

    except (Exception, psycopg2.DatabaseError) as error:
        print("insertRecordsAgain: ",error)

insertRecordsAgain(True)

Total Distinct Zones:  2
[('Data Prob', 'Data Prob.ipynb'), ('phish_score', 'phish_score.csv')]
Working on file  Data Prob.ipynb
{' Date': '7/8/2020 21:08', 'Score': 5.3, 'URL': 'http://sevenqbk.com/', 'IP': '103.26.204.158'}
createTableAgain:  syntax error at or near "Prob"
LINE 1: DROP TABLE IF EXISTS Data Prob CASCADE;
                                  ^

Inserting in  Data Prob
1
insertTableAgain:  'Date'
Working on file  phish_score.csv
{' Date': '7/8/2020 21:08', 'Score': 5.3, 'URL': 'http://sevenqbk.com/', 'IP': '103.26.204.158'}
phish_score table created. 
Inserting in  phish_score
1
insertTableAgain:  'Date'


In [38]:
#############################################################
allFilesAgain=getFilesAgain(False)
allFilesAgain

{'Data Prob': 'Data Prob.ipynb', 'phish_score': 'phish_score.csv'}

In [33]:
conn = psycopg2.connect(host=DB_HOST,database=DB_DATABASE, user=DB_USER, password=DB_PASS,sslmode="disable")
cur = conn.cursor()

In [34]:
################################################################
if allFilesAgain:
   # print(allFiles['Data Prob'])
    
    #    print("Working on file ",file[1])
    insertFileAgain("phish_score",allFilesAgain['phish_score'],cur,conn)

{' Date': '7/8/2020 21:08', 'Score': 5.3, 'URL': 'http://sevenqbk.com/', 'IP': '103.26.204.158'}
phish_score table created. 
Inserting in  phish_score
1
insertTableAgain:  'Date'


NameError: name 'connn' is not defined